In [1]:
import os 
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd # type: ignore

In [2]:
from utils import get_dataset, get_model_by_tag, get_prompt_template
from LLMAnnotator import LLMAnnotator

In [3]:
os.environ['OPENAI_API_KEY_CLARIN']=os.getenv("OPENAI_API_KEY_CLARIN")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

token = os.getenv("OPENAI_API_KEY_CLARIN")

In [4]:
import pandas as pd
import re

def extract_bracketed_word(df, read_col, write_col):
    """
    Wyodrębnia pierwszy wyraz objęty nawiasami [] z kolumny `read_col`
    i zapisuje go do kolumny `write_col`.
    
    :param df: DataFrame wejściowy
    :param read_col: Nazwa kolumny z której odczytywany jest tekst
    :param write_col: Nazwa kolumny do której zapisywany będzie wynik
    :return: DataFrame z nową kolumną
    """
    def extract_last_bracketed(text):
        matches = re.findall(r'\[([^\[\]]+?)\]', str(text))
        return matches[-1] if matches else None
    
    df[write_col] = df[read_col].apply(extract_last_bracketed)
    return df

In [5]:
import pandas as pd

def validate_text_column(path, df_reference):
    # Wczytaj DataFrame z pliku CSV
    df = pd.read_csv(path)
    
    # Pobierz unikalne wartości z kolumny 'text' z df_reference
    reference_texts = set(df_reference['text'].unique())
    
    # Dodaj kolumnę 'is_valid'
    df['was_in_selected_samples'] = df['text'].isin(reference_texts)
    df = extract_bracketed_word(df, 'output', 'extracted_label')
    # df.to_csv(path, index=False)
    
    return df

In [6]:
for i in [1]:
    print('')
    print('')
    print('MUMER', i)
    print('')
    print('')
    model_tag = 'llama3'  
    dataset_tag = '20_newsgroups'
    prompt = 'cot_random_samples_cohere'
    temp = 0
    number_of_experimet = i
    selected_samples =  f'{model_tag}_random42_80'
    
    
    model = get_model_by_tag(model_tag, token, temp)  
    dataset_for_annotation = get_dataset(dataset_tag=dataset_tag)[1350:]
    prompt_txt = get_prompt_template(dataset_tag, prompt)
    examples_for_prompt = pd.read_csv(f'./selected_samples/{dataset_tag}/{selected_samples}.csv')

    output_path = f'./resultsTEMP/{prompt}_temp{temp}/{model_tag}/{dataset_tag}_{model_tag}_{prompt}_{selected_samples}_temp{temp}_exp{number_of_experimet}.csv'
    os.makedirs(f'./resultsTEMP/{prompt}_temp{temp}/{model_tag}', exist_ok=True)
    
    
    text_col = "text"
    label_col = "label"
    output_col = "output"
    
    
    annotator = LLMAnnotator(
        model=model,
        dataset=dataset_for_annotation,
        examples_for_prompt=examples_for_prompt,
        prompt_template=prompt_txt,
        column_text=text_col,
        column_label=label_col,
        column_output=output_col
    )
    
    df = annotator.get_results(is_save=True, output_path=output_path)
    annotator.save_results(output_path=output_path)
    df_new = validate_text_column(output_path, examples_for_prompt)





MUMER 1




/home/julita/Desktop/anote-skrpyty 19.04/LLMAnnotator.py:77: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.chain = LLMChain(llm=self.model, prompt=self.prompt)


Nr: 0 Predicted label: To determine the most suitable category for the provided text, let's break down the steps as suggested:

**Step 1: Summarize the main topic of the text.**
The text revolves around a discussion about the Branch Davidians, specifically referencing their attire ("black guys in ninja suits") and questioning the media's portrayal or handling of this aspect, with implications of political correctness.

**Step 2: Identify any key terms or phrases that hint at a specific category.**
Key terms include "Branch Davidians," "media," "politically correct," and the context implies a discussion on a controversial event or group, potentially touching on themes of politics, social issues, or religion.

**Step 3: Compare the topic to the definitions of each category.**
Given the context, categories such as `soc.religion.christian`, `alt.atheism`, `talk.politics.misc`, `talk.politics.guns`, and `talk.religion.misc` could be potential matches because they all deal with discussions t

Failed to multipart ingest runs: Connection error caused failure to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. The content length of 223041389 bytes exceeds the maximum size limit of 20971520 bytes. SSLError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs/multipart (Caused by SSLError(SSLError(5, '[SYS] unknown error (_ssl.c:2417)')))"))
Content-Length: 223041389
API Key: lsv2_********************************************7etrace=718c263f-d06a-4419-8b60-2f2d04aae95b,id=718c263f-d06a-4419-8b60-2f2d04aae95b


In [7]:
df_new = validate_text_column(output_path, examples_for_prompt)
df_new

,text,output,logprobs,top_logprobs,original_label,error,was_in_selected_samples,extracted_label
0,article robert nehls writes kenneth whitehead ...,To determine the most suitable category for th...,"{'content': [{'token': 'To', 'logprob': -0.172...","[{'token': 'To', 'logprob': -0.172928631305694...",talk.politics.guns,NaN,False,Final Category: talk.politics.guns
1,article writes likely explanation may somethin...,To determine the most suitable category for th...,"{'content': [{'token': 'To', 'logprob': -0.179...","[{'token': 'To', 'logprob': -0.179383948445320...",comp.sys.mac.hardware,NaN,False,comp.sys.mac.hardware
2,holly ks wrote western digital also three sets...,To determine the most suitable category for th...,"{'content': [{'token': 'To', 'logprob': -0.186...","[{'token': 'To', 'logprob': -0.18692646920681}...",comp.sys.ibm.pc.hardware,NaN,False,comp.sys.ibm.pc.hardware
3,main advantages method seen described algiorit...,To determine the most suitable category for th...,"{'content': [{'token': 'To', 'logprob': -0.245...","[{'token': 'To', 'logprob': -0.245940238237380...",comp.graphics,NaN,False,comp.graphics
4,gerald olchowy writes bryan murray done little...,To determine the most suitable category for th...,"{'content': [{'token': 'To', 'logprob': -0.169...","[{'token': 'To', 'logprob': -0.169704392552375...",rec.sport.hockey,NaN,False,rec.sport.hockey
...,...,...,...,...,...,...,...,...
645,david sternlight writes article karl barrus wr...,To determine the most suitable category for th...,"{'content': [{'token': 'To', 'logprob': -0.181...","[{'token': 'To', 'logprob': -0.181033700704574...",sci.crypt,NaN,False,sci.crypt
646,hello im looking information alphanumeric page...,To determine the most suitable category for th...,"{'content': [{'token': 'To', 'logprob': -0.183...","[{'token': 'To', 'logprob': -0.183513775467872...",sci.electronics,NaN,False,sci.electronics
647,john r daker writes would like offocially nomi...,To determine the most suitable category for th...,"{'content': [{'token': 'To', 'logprob': -0.148...","[{'token': 'To', 'logprob': -0.148061156272888...",rec.motorcycles,NaN,False,rec.motorcycles
648,article writes looking information concerning ...,To determine the most suitable category for th...,"{'content': [{'token': 'To', 'logprob': -0.268...","[{'token': 'To', 'logprob': -0.268602669239044...",sci.space,NaN,False,sci.space


In [8]:
df = pd.read_csv(output_path)
df

,text,output,logprobs,top_logprobs,original_label,error
0,article robert nehls writes kenneth whitehead ...,To determine the most suitable category for th...,"{'content': [{'token': 'To', 'logprob': -0.172...","[{'token': 'To', 'logprob': -0.172928631305694...",talk.politics.guns,NaN
1,article writes likely explanation may somethin...,To determine the most suitable category for th...,"{'content': [{'token': 'To', 'logprob': -0.179...","[{'token': 'To', 'logprob': -0.179383948445320...",comp.sys.mac.hardware,NaN
2,holly ks wrote western digital also three sets...,To determine the most suitable category for th...,"{'content': [{'token': 'To', 'logprob': -0.186...","[{'token': 'To', 'logprob': -0.18692646920681}...",comp.sys.ibm.pc.hardware,NaN
3,main advantages method seen described algiorit...,To determine the most suitable category for th...,"{'content': [{'token': 'To', 'logprob': -0.245...","[{'token': 'To', 'logprob': -0.245940238237380...",comp.graphics,NaN
4,gerald olchowy writes bryan murray done little...,To determine the most suitable category for th...,"{'content': [{'token': 'To', 'logprob': -0.169...","[{'token': 'To', 'logprob': -0.169704392552375...",rec.sport.hockey,NaN
...,...,...,...,...,...,...
645,david sternlight writes article karl barrus wr...,To determine the most suitable category for th...,"{'content': [{'token': 'To', 'logprob': -0.181...","[{'token': 'To', 'logprob': -0.181033700704574...",sci.crypt,NaN
646,hello im looking information alphanumeric page...,To determine the most suitable category for th...,"{'content': [{'token': 'To', 'logprob': -0.183...","[{'token': 'To', 'logprob': -0.183513775467872...",sci.electronics,NaN
647,john r daker writes would like offocially nomi...,To determine the most suitable category for th...,"{'content': [{'token': 'To', 'logprob': -0.148...","[{'token': 'To', 'logprob': -0.148061156272888...",rec.motorcycles,NaN
648,article writes looking information concerning ...,To determine the most suitable category for th...,"{'content': [{'token': 'To', 'logprob': -0.268...","[{'token': 'To', 'logprob': -0.268602669239044...",sci.space,NaN


In [12]:
output_path2 = f'./results/{prompt}_temp{temp}/{model_tag}/{dataset_tag}_{model_tag}_{prompt}_{selected_samples}_temp{temp}_exp{number_of_experimet}.csv'
df_old = pd.read_csv(output_path2)
df_old = validate_text_column(f'./results/{prompt}_temp{temp}/{model_tag}/{dataset_tag}_{model_tag}_{prompt}_{selected_samples}_temp{temp}_exp{number_of_experimet}.csv', examples_for_prompt) 


In [16]:
df_combined = pd.concat([df_old, df_new])

In [22]:
df_combined.to_csv(output_path2)